In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
import sqlite3
import pandas as pd
import numpy as np
import nltk
import string

In [5]:
conn = sqlite3.connect('/content/drive/My Drive/Senti/database.sqlite')

In [6]:
data = pd.read_sql_query(''' SELECT * FROM Reviews WHERE Score !=3''',conn)

In [7]:
type(data)

pandas.core.frame.DataFrame

In [8]:
score_num=list(data['Score'])

In [9]:
Polarity=[]
for i in score_num:
  if i > 3:
    Polarity.append(1)
  else:
    Polarity.append(0)


In [10]:
data['polarity']=Polarity

In [11]:
data['polarity'].value_counts()

1    443777
0     82037
Name: polarity, dtype: int64

In [12]:
new_data=data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)

In [13]:
new_data=new_data[new_data.HelpfulnessNumerator<=new_data.HelpfulnessDenominator]

In [14]:
new_data['polarity'].value_counts()

1    307061
0     57110
Name: polarity, dtype: int64

In [15]:
yes=new_data[new_data['polarity']== 1]
no=new_data[new_data['polarity']== 0]

In [16]:
yes.shape

(307061, 11)

In [17]:
yess = yes.iloc[:50000]
noo = no.iloc[:50000]

In [18]:
df=yess.append(noo)

In [19]:
df.shape

(100000, 11)

In [20]:
df = df.sample(frac=1).reset_index(drop=True)

In [21]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,polarity
0,28673,B003EH6P9Q,A2QNVYNMG56DTF,Bender,0,4,1,1331424000,Price,"I really like this coffee but,I'll be dammed i...",0
1,429162,B0018CIP6K,A28D1YLEHYD1U6,KDK,3,5,1,1336176000,FOOD RECALL,Taste of the Wild has recently issued a recall...,0
2,10053,B0007ZLRUU,A3R71YQ3WXBZTO,W. Taylor,2,4,1,1230249600,Only 2 of 3 16 oz. received in shipment,"I recently ordered the 16 oz., 3 Pack. When m...",0
3,38258,B0085HLZ4K,A1UHL2T5NLT7QE,MJM,0,0,4,1341964800,Almost like McDonalds' McCafe,"Pretty good coffee - good flavor, although it ...",1
4,119523,B0041LHND6,A1SFKK0DIH2JKQ,Barbara,35,38,1,1273449600,BEWARE,We've had several nylabones for our dogs. The...,0


In [22]:
df['polarity'].value_counts()

1    50000
0    50000
Name: polarity, dtype: int64

In [23]:
import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [24]:
#!pip install nltk

In [25]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [26]:
stop = set(stopwords.words('english'))
lemma=nltk.stem.WordNetLemmatizer()

In [27]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [28]:
preprocessed_reviews = []
# tqdm is for printing the status bar
for sentance in (df['Text'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    # sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_reviews.append(sentance.strip())

In [29]:
df['cleanedText'] = preprocessed_reviews
df['cleanedText'] = df['cleanedText'].astype('str')

In [30]:
import seaborn as sns
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [31]:
x=df['cleanedText']
y=df['polarity']

In [32]:
from sklearn.model_selection import train_test_split

In [33]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30,random_state=355)

In [34]:
from keras.preprocessing.text import Tokenizer

In [35]:
#new_data['CleanedText'] = new_data['CleanedText'].astype('str')

In [36]:
#type(x_train[0])

In [37]:
tokenize = Tokenizer(num_words=5000)
tokenize.fit_on_texts(x_train)

X_train_new = tokenize.texts_to_sequences(x_train)
X_test_new = tokenize.texts_to_sequences(x_test)

In [38]:
import keras
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.models import Sequential
from keras.preprocessing import sequence

In [39]:
max_review_length = 600
X_train_new = sequence.pad_sequences(X_train_new, maxlen=max_review_length)
X_test_new = sequence.pad_sequences(X_test_new, maxlen=max_review_length)


In [40]:
#from keras.callbacks import EarlyStopping

In [41]:
embed_vector_length = 32
model = Sequential()
model.add(Embedding(5000, embed_vector_length, input_length=max_review_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#monitor = EarlyStopping(monitor='val_loss',min_delta=0,mode='auto',restore_best_weights=True)

In [42]:
 model.fit(X_train_new, y_train, epochs=10,
                   batch_size=128, verbose=1, validation_data=(X_test_new,y_test))

Epoch 1/10
547/547 [==============================] - 137s 251ms/step - loss: 0.3368 - accuracy: 0.8527 - val_loss: 0.2742 - val_accuracy: 0.8879
Epoch 2/10
547/547 [==============================] - 137s 250ms/step - loss: 0.2478 - accuracy: 0.9033 - val_loss: 0.2581 - val_accuracy: 0.8976
Epoch 3/10
547/547 [==============================] - 137s 251ms/step - loss: 0.2245 - accuracy: 0.9142 - val_loss: 0.2608 - val_accuracy: 0.8983
Epoch 4/10
547/547 [==============================] - 137s 251ms/step - loss: 0.2025 - accuracy: 0.9228 - val_loss: 0.2653 - val_accuracy: 0.8988
Epoch 5/10
547/547 [==============================] - 138s 252ms/step - loss: 0.1826 - accuracy: 0.9323 - val_loss: 0.2784 - val_accuracy: 0.8988
Epoch 6/10
547/547 [==============================] - 137s 250ms/step - loss: 0.1641 - accuracy: 0.9391 - val_loss: 0.2552 - val_accuracy: 0.9000
Epoch 7/10
547/547 [==============================] - 137s 251ms/step - loss: 0.1458 - accuracy: 0.9471 - val_loss: 0.2770 -

In [67]:
import tensorflow

In [68]:
from tensorflow.keras.models import

In [99]:
model.save('/content/drive/My Drive/Senti/senment.h5')

In [52]:
!pip install PyAudio

  ERROR: Failed building wheel for PyAudio
  Running setup.py clean for PyAudio
Failed to build PyAudio
    Running setup.py install for PyAudio ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-qlesc4nb/PyAudio/setup.py'"'"'; __file__='"'"'/tmp/pip-install-qlesc4nb/PyAudio/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-vorbeqx5/install-record.txt --single-version-externally-managed --compile Check the logs for full command output.


In [112]:
instance = ["don't buy this and it is fake"]

In [113]:
preprocessed_reviews = []
# tqdm is for printing the status bar
for sentance in (instance):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    # sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_reviews.append(sentance.strip())

In [114]:
instance = tokenize.texts_to_sequences(preprocessed_reviews)
flat_list=[]
for sublist in instance:
  for item in sublist:
    flat_list.append(item)

flat_list= [flat_list]
instance = sequence.pad_sequences(flat_list, maxlen=max_review_length)

In [115]:
res=model.predict(instance)
print(res)
if res > 0.5:
  print("pos")
else:
  print("neg")

[[0.03295285]]
neg


In [61]:
!pip install SpeechRecognition

     |████████████████████████████████| 32.8MB 90kB/s 


In [ ]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

In [ ]:
!pip install pyaudio

In [62]:
import speech_recognition as sr


In [63]:
r = sr.Recognizer()


In [65]:
!pip install ffmpeg-python

In [66]:
with sr.Microphone() as source:
  print("speak anything....")
  audio = r.listen(source)
  try:
    text = r.recognize_google(audio)
    print("you said .. : ",text)
  except:
    print("sorry could not hear....")
    

OSError: ignored

In [141]:
!pip install PyAudio

  ERROR: Failed building wheel for PyAudio
  Running setup.py clean for PyAudio
Failed to build PyAudio
    Running setup.py install for PyAudio ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-ss3rtvrl/PyAudio/setup.py'"'"'; __file__='"'"'/tmp/pip-install-ss3rtvrl/PyAudio/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-i5o_1lh8/install-record.txt --single-version-externally-managed --compile Check the logs for full command output.


In [47]:
import tensorflow
tensorflow.__version__

'2.3.0'

In [53]:
!pip install pyaudio


  Using cached https://files.pythonhosted.org/packages/ab/42/b4f04721c5c5bfc196ce156b3c768998ef8c0ae3654ed29ea5020c749a6b/PyAudio-0.2.11.tar.gz
  ERROR: Failed building wheel for pyaudio
  Running setup.py clean for pyaudio
Failed to build pyaudio
    Running setup.py install for pyaudio ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-qti0bbm2/pyaudio/setup.py'"'"'; __file__='"'"'/tmp/pip-install-qti0bbm2/pyaudio/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-pmn045uy/install-record.txt --single-version-externally-managed --compile Check the logs for full command output.


In [58]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libasound2-dev is already the newest version (1.1.3-5ubuntu0.5).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 184 kB of archives.
After this operation, 891 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudiocpp0 amd64 19.6.0-1 [15.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 portaudio19-dev amd64 19.6.0-1 [104 kB]
Fetched 184 kB in 2s (102 kB/s)
Selecting pr

In [59]:
!pip install pyaudio

  Using cached https://files.pythonhosted.org/packages/ab/42/b4f04721c5c5bfc196ce156b3c768998ef8c0ae3654ed29ea5020c749a6b/PyAudio-0.2.11.tar.gz
  Created wheel for pyaudio: filename=PyAudio-0.2.11-cp36-cp36m-linux_x86_64.whl size=51616 sha256=ff92151943ef59dcbdddc098a677b2a1994cea717aad64bb58eb2004fdb08c7d
  Stored in directory: /root/.cache/pip/wheels/f4/a8/a4/292214166c2917890f85b2f72a8e5f13e1ffa527c4200dcede
Successfully built pyaudio


In [49]:
from keras.models import load_model

In [51]:
mod= load_model('/content/drive/My Drive/Senti/senti.h5')

In [72]:
print(tensorflow.__version__)

2.3.0


# New Section